# Sort Query Results
#### (WDS objects have been queried in Gaia, now it's time to sort them based on parallax, separation, and proper motion)
#### These methods origniated in the el Badry et al publication in 2021 - implemented by Daphne Zakarian

In [ ]:
from astropy.io import ascii
from astropy.table import vstack, Table, unique, QTable
from astropy.coordinates import SkyCoord 
import astropy.units as u
from astropy import table, log
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Distance, Angle
from astropy.time import Time

import os
from IPython.display import display
from multiprocessing import set_start_method

from math import sqrt, cos

## Read in the stacked results table and assign units to the columns

In [ ]:
# google colab read file in (google colab): 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# after mounting drive, copy the path of the directory where the files are stored
save_path='/content/drive/MyDrive/NOFS copy/WiserLabBackup/QueryResults'

In [ ]:
# # Read the file jupyter lab

# # vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# # wiser's lab comp:
# save_path = '/home/student/djz7128/djz_NOFS/QueryResults'


In [ ]:
# open files:

qrt ='{0}/stack_query_results_table.ecsv'.format(save_path) 
ie = '{0}/stack_index_error_queries.ecsv'.format(save_path)
query_results_table = QTable.read(qrt, header_start=0, data_start=1)
index_error_queries = Table.read(ie, header_start=0, data_start=1)


## Assign Units to the table columns: 

## DEGREES 

deg_columns = ['ra_a', 'ra_b', 'dec_a', 'dec_b']
for deg_column in deg_columns:
    query_results_table[deg_column] = query_results_table[deg_column]*u.deg
    
## MAS

mas_columns = ['ra_error_a', 'ra_error_b', 'dec_error_a', 'dec_error_b', 'parallax_a', 'parallax_b', 'parallax_error_a', 'parallax_error_b']
for mas_column in mas_columns:
    query_results_table[mas_column] = query_results_table[mas_column]*u.mas

## MAS/YR
mas_p_year_columns = ['pmra_a', 'pmra_b', 'pmra_error_a', 'pmra_error_b', 'pmdec_a', 'pmdec_b', 'pmdec_error_a', 'pmdec_error_b']
for mas_p_year_column in mas_p_year_columns:
    query_results_table[mas_p_year_column] = query_results_table[mas_p_year_column]*u.mas/u.yr

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486314701,0.008348978124558926,0.014539013616740704,2.384840253174304,2.3315141654599225,0.013162785209715366,0.023133113980293274,181.18052673339844,100.78687286376953,-0.9147508384598348,-1.064158531848239,0.010332660749554634,0.01920725405216217,-8.803328013644308,-8.914049677675674,0.011068823747336864,0.01896110363304615,-3.078555107116699,0.7382858991622925,1.3963912725448608,1.2604676485061646

In [ ]:
# add calculated columns(some will be updated in the loop)

# SEPARATION

# the mu columns are all proper motion calculations
query_results_table.add_column(0.0*u.arcsec, name='separation')
query_results_table.add_column(0.0*u.mas/u.yr, name='delta_mu_orbit')
query_results_table.add_column(0.0*u.mas**2/u.yr**2, name='delta_mu_ra2')
query_results_table.add_column(0.0*u.mas**2/u.yr**2, name='delta_mu_dec2')
query_results_table.add_column(0.0*u.mas/u.yr, name='delta_mu')
query_results_table.add_column(0.0*u.mas/u.yr, name='sigma_delta_mu')


for rownum in range(len(query_results_table)):
    ra_a, ra_b = query_results_table[rownum]['ra_a'], query_results_table[rownum]['ra_b']
    dec_a, dec_b = query_results_table[rownum]['dec_a'], query_results_table[rownum]['dec_b']
    # use the ra and dec of component a and b to make SkyCoord objects
    coord_a = SkyCoord(ra_a, dec_a, frame='icrs')
    coord_b = SkyCoord(ra_b, dec_b, frame='icrs')

    # calcualte the separation between the two objects
    separation = coord_a.separation(coord_b).to(u.arcsec)

    query_results_table['separation'][rownum] = separation
    if rownum%5000==0:
        print(rownum)

 

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [ ]:
query_results_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292,6.593528748410601,0.0,0.0,0.0,0.0,0.0
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429,8.13322312188097,0.0,0.0,0.0,0.0,0.0
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486314701,0.008348978124558926,0.014539013616740704,2.384840253174304,2.3315141654599225,0.013162785209715366,0.023133113980293274,181.180526733398

## Create output tables: associated systems and non-associated systems
#### I can't find a simple way to copy astropy tables so I'll just reopen the query results table and assign units like before
#### I think assigning a new name still refers to the og table... whatever was causing the issue, this way works fine!!


I will start with the non-associated systems table being identical to the query_results_table.
As I loop the results table, the row number of the non-associated and associated systems will be appended to a list.
After the loop, I will use these row numbers to copy the associated systems to their output table, 
and also to remove those rows from the non-associated systems table.

In [ ]:
# Read the file

# ## vayu's lab comp:
# # save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# # wiser's lab comp:
# save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

# qrt ='{0}/stack_query_results_table.ecsv'.format(save_path) 


associated_systems_table = QTable.read(qrt, header_start=0, data_start=1)
non_associated_systems_table = QTable.read(qrt, header_start=0, data_start=1)








## Assign Units: 

## DEGREES 

deg_columns = ['ra_a', 'ra_b', 'dec_a', 'dec_b']
for deg_column in deg_columns:
    associated_systems_table[deg_column] = associated_systems_table[deg_column]*u.deg
    non_associated_systems_table[deg_column] = non_associated_systems_table[deg_column]*u.deg
## MAS

mas_columns = ['ra_error_a', 'ra_error_b', 'dec_error_a', 'dec_error_b', 'parallax_a', 'parallax_b', 'parallax_error_a', 'parallax_error_b']
for mas_column in mas_columns:
    associated_systems_table[mas_column] = associated_systems_table[mas_column]*u.mas
    non_associated_systems_table[mas_column] = non_associated_systems_table[mas_column]*u.mas
## MAS/YR
mas_p_year_columns = ['pmra_a', 'pmra_b', 'pmra_error_a', 'pmra_error_b', 'pmdec_a', 'pmdec_b', 'pmdec_error_a', 'pmdec_error_b']
for mas_p_year_column in mas_p_year_columns:
    associated_systems_table[mas_p_year_column] = associated_systems_table[mas_p_year_column]*u.mas/u.yr
    non_associated_systems_table[mas_p_year_column] = non_associated_systems_table[mas_p_year_column]*u.mas/u.yr
    

# add calculated columns
# separation calcualted ahead of time, proper motion info (all of the mu and sigmas) are calculated in the loop


associated_systems_table.add_column(0*u.arcsec, name='separation')
associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu_orbit')
associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_ra2')
associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_dec2')
associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu')
associated_systems_table.add_column(0*u.mas/u.yr, name='sigma_delta_mu')

# for rownum in range(len(associated_systems_table)):
#     ra_a, ra_b = associated_systems_table[rownum]['ra_a'], associated_systems_table[rownum]['ra_b']
#     dec_a, dec_b = associated_systems_table[rownum]['dec_a'], associated_systems_table[rownum]['dec_b'] 
#     # use the ra and dec of component a and b to make SkyCoord objects
#     coord_a = SkyCoord(ra_a, dec_a, frame='icrs')
#     coord_b = SkyCoord(ra_b, dec_b, frame='icrs')

#     # calcualte the separation between the two objects
#     separation = coord_a.separation(coord_b)
#     associated_systems_table['separation'][rownum] = separation
    
#     if rownum%5000==0:
#         print(rownum)
    

non_associated_systems_table.add_column(0*u.arcsec, name='separation')
non_associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu_orbit')
non_associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_ra2')
non_associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_dec2')
non_associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu')
non_associated_systems_table.add_column(0*u.mas/u.yr, name='sigma_delta_mu')


for rownum in range(len(query_results_table)):
    ra_a, ra_b = non_associated_systems_table[rownum]['ra_a'], non_associated_systems_table[rownum]['ra_b']
    dec_a, dec_b = non_associated_systems_table[rownum]['dec_a'], non_associated_systems_table[rownum]['dec_b'] 
    # use the ra and dec of component a and b to make SkyCoord objects
    coord_a = SkyCoord(ra_a, dec_a, frame='icrs')
    coord_b = SkyCoord(ra_b, dec_b, frame='icrs')

    # calcualte the separation between the two objects
    separation = coord_a.separation(coord_b)
    non_associated_systems_table['separation'][rownum] = separation
    if rownum%5000==0:
      print(rownum)
    
## Remove the rows from the associated_systems_table
table_length = len(associated_systems_table)
associated_systems_table.remove_rows(slice(0,table_length))

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [ ]:
associated_systems_table # this should be empty 

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


## LOOP THROUGH RESULTS TABLE AND RECORD WHICH ROWS ARE NON-ASSOCIATED PAIRS

In [ ]:
# ## testing part of the loop

# rownum = 0

# """ READ IN THE RELEVANT VALUES:  ra, dec, parallax, pmra, pmdec, g magnitude, parallax uncertainties, proper motion uncertainties """
# ra_a, ra_b = query_results_table[rownum]['ra_a'], query_results_table[rownum]['ra_b']
# dec_a, dec_b = query_results_table[rownum]['dec_a'], query_results_table[rownum]['dec_b']
# parallax_a, parallax_b = query_results_table[rownum]['parallax_a'], query_results_table[rownum]['parallax_b']
# pmra_a, pmra_b = query_results_table[rownum]['pmra_a'], query_results_table[rownum]['pmra_b']
# pmdec_a, pmdec_b = query_results_table[rownum]['pmdec_a'], query_results_table[rownum]['pmdec_a']
# phot_g_mean_mag_a, phot_g_mean_mag_b = query_results_table[rownum]['phot_g_mean_mag_a'], query_results_table[rownum]['phot_g_mean_mag_b']
# parallax_error_a, parallax_error_b = query_results_table[rownum]['parallax_error_a'], query_results_table[rownum]['parallax_error_b']
# pmra_error_a, pmra_error_b = query_results_table[rownum]['pmra_error_a'], query_results_table[rownum]['pmra_error_b']
# pmdec_error_a, pmdec_error_b = query_results_table[rownum]['pmdec_error_a'], query_results_table[rownum]['pmdec_error_a']
# separation = query_results_table[rownum]['separation']


# """ ASTROMETRIC CUT 1) PROJECTED SEPARATIONS """


# # figure out which component has brighter g mag and indicate brighter object's parallax
# # if one of the mags is missing, assume component a is brighter
# if phot_g_mean_mag_a > phot_g_mean_mag_b: # larger mag -> dimmer object
#     brighter_component_parallax = parallax_b
# else:
#     brighter_component_parallax = parallax_a
    
# # if the separation is too large, then the pair is not associated
# if separation > 206.25*brighter_component_parallax:
#     non_associated_rownum.append(rownum)
    
    

    

In [ ]:
len(non_associated_systems_table)

In [ ]:
""" INITIALIZE non_associated_rownum AND associated_systems_rownum LISTS TO KEEP TRACK OF OBJECTS THAT DO NOT PASS ANY ONE OF THE CUTS """

non_associated_rownum = []
associated_systems_rownum = []



""" Go through query results table and sort objects as associated or non-associated """
for rownum in range(len(query_results_table)):

    if rownum%5000==0:
        print(rownum)

    """ READ IN THE RELEVANT VALUES:  ra, dec, parallax, pmra, pmdec, g magnitude, parallax uncertainties, proper motion uncertainties """
    ra_a, ra_b = query_results_table[rownum]['ra_a'], query_results_table[rownum]['ra_b']
    dec_a, dec_b = query_results_table[rownum]['dec_a'], query_results_table[rownum]['dec_b']
    parallax_a, parallax_b = query_results_table[rownum]['parallax_a'], query_results_table[rownum]['parallax_b']
    pmra_a, pmra_b = query_results_table[rownum]['pmra_a'], query_results_table[rownum]['pmra_b']
    pmdec_a, pmdec_b = query_results_table[rownum]['pmdec_a'], query_results_table[rownum]['pmdec_b']
    phot_g_mean_mag_a, phot_g_mean_mag_b = query_results_table[rownum]['phot_g_mean_mag_a'], query_results_table[rownum]['phot_g_mean_mag_b']
    parallax_error_a, parallax_error_b = query_results_table[rownum]['parallax_error_a'], query_results_table[rownum]['parallax_error_b']
    pmra_error_a, pmra_error_b = query_results_table[rownum]['pmra_error_a'], query_results_table[rownum]['pmra_error_b']
    pmdec_error_a, pmdec_error_b = query_results_table[rownum]['pmdec_error_a'], query_results_table[rownum]['pmdec_error_a']
    separation = query_results_table[rownum]['separation']

    # proper motion calculations:
    
    # figure out which component has brighter g mag and indicate brighter object's parallax
    # if one of the mags is missing, assume component a is brighter
    if phot_g_mean_mag_a > phot_g_mean_mag_b: # larger mag -> dimmer object
        brighter_component_parallax = parallax_b
    else:
        brighter_component_parallax = parallax_a
    
    # delta_mu_orbit is the max proper motion difference expected for orbital motion (depending on given sep and parallax)
    delta_mu_orbit = ( 0.44*(brighter_component_parallax/u.mas)**(3/2) * (separation.to(u.arcsec)/u.arcsec)**(1/2) )*u.mas/u.yr
    
    # mu_star is pmra * cos(dec)
    # note: the error in ra is really the error of this value with cosine included
    # another note: can't do cos of units
    mu_star_ra_a = pmra_a/u.mas * cos(dec_a/u.mas)
    mu_star_ra_b = pmra_b/u.mas * cos(dec_b/u.mas)
    mu_star_ra_a = mu_star_ra_a*u.mas
    mu_star_ra_b = mu_star_ra_b*u.mas
    
    # delta_mu_ra2 is the square of the difference of mu_star_a - mu_star_b
    delta_mu_ra2 = (mu_star_ra_a - mu_star_ra_b)**2
    
    # delta_mu_dec2 is the square of the difference of pmdec_a - pmdec_b
    delta_mu_dec2 = (pmdec_a - pmdec_b)**2
        
    # delta_mu is the scalar proper motion difference, which is:
    # [ (mu_star_ra_a - mu_star_ra_b)^2 +(pmdec_a - pmdec_b)^2 ]^(1/2)
    delta_mu = ( (mu_star_ra_a - mu_star_ra_b)**2 + (pmdec_a - pmdec_b)**2 )**(1/2)
    

    # sigma_delta_mu is (I believe) the weighted uncertainty of delta_mu
    sigma_delta_mu = (1/delta_mu) * ( (pmra_error_a**2 + pmra_error_b**2) * delta_mu_ra2 + (pmdec_error_a**2 + pmdec_error_b**2) * delta_mu_dec2 )**(1/2)
    
    
    # update these pm calculations in the query results table and associated systems table
    query_results_table['delta_mu_orbit'][rownum] = delta_mu_orbit
    query_results_table['delta_mu_ra2'][rownum] = delta_mu_ra2
    query_results_table['delta_mu_dec2'][rownum] = delta_mu_dec2
    query_results_table['delta_mu'][rownum] = delta_mu
    query_results_table['sigma_delta_mu'][rownum] = sigma_delta_mu
    
    
    non_associated_systems_table['delta_mu_orbit'][rownum] = delta_mu_orbit
    non_associated_systems_table['delta_mu_ra2'][rownum] = delta_mu_ra2
    non_associated_systems_table['delta_mu_dec2'][rownum] = delta_mu_dec2
    non_associated_systems_table['delta_mu'][rownum] = delta_mu
    non_associated_systems_table['sigma_delta_mu'][rownum] = sigma_delta_mu
    
    
    
    """ ASTROMETRIC CUT 1) PROJECTED SEPARATIONS """
    # if the 2-d separation in the sky is too large, the pair is not physically associated
    # so, we should add this row number to the list of rows that are not associated, and then move on to the next rownum
    if separation/u.arcsec > 206.25*brighter_component_parallax/u.mas:
        non_associated_rownum.append(rownum)
        continue
        
    # if the separation is close enough, we move on to the next round of cuts
    
    """ ASTROMETRIC CUT 2) PARALLAX SIMILARITY """
    # If the absolute value of the differnce of the parallax is too large, then the 3-d separation is too large,
    # therefore, the pair is not associated
    # This cut-off depends on the separation
    
    # absolute value of the difference of the parallaxes
    parallax_diff = abs(parallax_a - parallax_b)
    
    # add the uncertainties of the parallaxes in quadrature
    # note that there can't be units in the square root
    parallax_error_quad = sqrt((parallax_error_a**2 + parallax_error_b**2)/u.mas**2)
    parallax_error_quad = parallax_error_quad*u.mas
    
    # set the value of b depending on the separation
    if separation > 4*u.arcsec:
        b = 3
    elif separation <= 4*u.arcsec:
        b = 6
        
    # if the parallax diff is too large, the pair is not associated
    # so, we should add this row number to the list of rows that are not associated, and then move on to the next rownum
    if parallax_diff > b * parallax_error_quad:
        non_associated_rownum.append(rownum)
        continue
    
    """ ASTROMETRIC CUT 3) PROPER MOTION SIMILARITY """
    # now if the proper motions aren't sufficiently similar, those pairs are not associated


    
    # if the pm difference is too large, the pair isn't physically associated
    if delta_mu >= delta_mu_orbit + 2 * sigma_delta_mu:
        non_associated_rownum.append(rownum)
        continue
    
    else:   
      associated_systems_rownum.append(rownum) 
 

0
5000


/home/student/djz7128/.conda/envs/djzconda/lib/python3.11/site-packages/astropy/units/quantity.py:611: RuntimeWarning: divide by zero encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/home/student/djz7128/.conda/envs/djzconda/lib/python3.11/site-packages/astropy/units/quantity.py:611: RuntimeWarning: invalid value encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [ ]:
len(associated_systems_rownum)

In [ ]:
# finalize the rows in associated and non-associated tables     
        
associated_systems_table_rownum = 0


for rownum in reversed(associated_systems_rownum):


    if rownum%100==0:
        print(rownum)

    non_associated_systems_table.remove_row(rownum)
    associated_systems_table.add_row()
    associated_systems_table[associated_systems_table_rownum] = query_results_table[rownum]
    associated_systems_table_rownum +=1


106800
106400
105900
105800
105700
105600
105100
104900
104800
104700
104100
104000
103400
103200
102500
102400
102200
101900
101000
100500
99600
98300
98000
97700
97500
96700
96200
95100
94900
94300
93600
93400
93300
92900
92600
91700
91200
90800
90600
90500
90100
89400
89100
89000
88900
88400
87800
87700
87500
86800
86600
86200
86100
85900
85800
85600
85500
85300
84900
84700
84000
83800
83600
83500
83000
82400
82200
82000
81900
81400
81200
81100
81000
80800
80700
79900
79800
79600
79500
79400
79300
79200
78900
78300
78000
77900
77400
77200
77100
77000
76900
76600
76400
76100
76000
75900
75700
75600
75200
75000
74000
73600
73500
73400
73200
72900
72600
71900
71700
71500
71100
71000
70900
70400
70100
69000
68500
68300
68100
68000
67700
67600
67500
66900
66500
65800
65500
65000
64900
64800
64500
64300
64000
63800
63700
63600
63000
62700
62400
62300
61900
61700
61500
61300
60400
60000
59900
59800
59600
59400
59200
59000
58700
58400
58200
58100
58000
57400
57200
56900
56800
56700
56300
55

In [ ]:
associated_systems_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
--,0,--,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan
23599-0412CBL,154510,2447741345626066816,2447741414345542912,2016.0,2016.0,359.98910273713295,359.97866966531234,0.044518087059259415,0.03720077872276306,-4.222556319237097,-4.203852018377835,0.03275608643889427,0.02752946875989437,7.341587718785455,7.35690441227576,0.05945796146988869,0.05121235176920891,123.47526550292969,143.65487670898438,-50.914530329842016,-51.0497075734875,0.05028996989130974,0.04505782201886177,-96.61254408846709,-96.83993825765693,0.035345956683158875,0.029300067573785782,nan,-1.6464951038360596,nan,9.40335750579834,31.0,31.0,15.0,15.0,0.07105975598096848,0.06368912011384964,0.9988299012184143,1.1016029119491577,15.876060485839844,15.31973648071289,1799.7384033203125,2187.729736328125,17.351524353027344,16.692827224731445,167.6161346435547,216.22621154785156,14.682077407836914,14.162753105163574,646.338134765625,727.8319702148438,2.6694469451904297,2.530074119567871,1.4504989385604858,1.4265681505203247,77.05281251384164,77.07075159620221,3385.2978385225424,0.051708108181535664,58.183756724972,0.06752226982816444
23599+1413SKF,154508,2767453557178693504,2767453557180534784,2016.0,2016.0,359.98521541821066,359.98514367073517,0.037190500646829605,0.03587879613041878,14.216942479375586,14.217371899389931,0.025418229401111603,0.02432810328900814,1.6240396632633158,1.5957870668654368,0.04658302664756775,0.04530474171042442,34.86333465576172,35.2234001159668,-2.4962877740722207,-2.748245912554472,0.046957191079854965,0.04558112099766731,-0.5499944481795733,0.09598252165938462,0.027971575036644936,0.026745609939098358,nan,nan,nan,nan,95.0,95.0,16.0,16.0,0.06524630635976791,0.06335528194904327,1.0669173002243042,1.1005388498306274,15.820831298828125,15.545248985290527,328.1726989746094,1095.4854736328125,nan,16.060087203979492,nan,36.727745056152344,nan,14.258393287658691,nan,85.24205017089844,nan,1.8016939163208008,nan,1.8284350633621216,1.566056890781894,1.109989461139578,0.009367756645182246,0.41728624556232197,0.653187570463113,0.040304973657372414
23599+2353DVG,154507,2848

In [ ]:
## vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# wiser's lab comp:
save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

# write the output files to end in _c# where # is the core number that was used

ascii.write(associated_systems_table, '{path}/associated_systems_table.ecsv'.format(path = save_path), format='ecsv') # overwrite = true
ascii.write(associated_systems_table, '{path}/associated_systems_table.csv'.format(path = save_path), format='csv')

ascii.write(non_associated_systems_table, '{path}/non_associated_systems_table.ecsv'.format(path = save_path), format='ecsv')
ascii.write(non_associated_systems_table, '{path}/non_associated_systems_table.csv'.format(path = save_path), format='csv')

In [ ]:
associated_systems_table

In [ ]:
non_associated_systems_table

In [ ]:
query_results_table